<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">



## Declare Factory

In [1]:
TMVA::Tools::Instance();

TFile *inputFile = TFile::Open( "inputdata.root");
TFile *outputFile = TFile::Open("TMVAOutputCV.root", "RECREATE" );

TMVA::Factory *factory = new TMVA::Factory("TMVAClassification", outputFile,
                             "!V:!ROC:!Correlations:Silent:Color:!DrawProgressBar:AnalysisType=Classification" ); 

## Declare DataLoader

In [2]:
TMVA::DataLoader *loader=new TMVA::DataLoader("dataset");

loader->AddVariable( "var1", 'F' );
loader->AddVariable( "var2", 'F' );
loader->AddVariable( "var3", 'F' );
loader->AddVariable( "var4", 'F' );
loader->AddVariable( "var5 := var1-var2", 'F' );
loader->AddVariable( "var6 := var1*var2", 'F' );

## Setup Dataset

In [3]:
TTree *tsignal     = (TTree*)inputFile->Get("Sig");
TTree *tbackground = (TTree*)inputFile->Get("Bkg");

TCut mycuts = "";
TCut mycutb = "";
   
loader->AddSignalTree    ( tsignal,     1.0 ); //signal weight     = 1
loader->AddBackgroundTree( tbackground, 1.0 ); //background weight = 1      
loader->PrepareTrainingAndTestTree( mycuts, mycutb,
"nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V" ); 

## Perform Cross Validation

In [4]:
factory->CrossValidate(loader, TMVA::Types::kBDT,  "BDT","!V:NTrees=10:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20", /*optimise hyper-parameters*/ false, /*Number of folds*/ 5);

Fold 1 ROCIntegral: 0.849132
Fold 2 ROCIntegral: 0.84909
Fold 3 ROCIntegral: 0.854479
Fold 4 ROCIntegral: 0.872519
Fold 5 ROCIntegral: 0.860324
Average ROCIntegral: 0.857109


## Cleanup memory

In [5]:
outputFile->Close();
delete factory;
delete loader;